In [9]:

import torch 
import sys

sys.path.append('..')

from torch import nn 
from torch.nn import functional as F
from torch import optim
from utils.loader import load

# load the dataset

train_input,train_target, train_classes, test_input, test_target, test_classes = load()
train_target=train_target.float()


# 2 channel net

class Net2(nn.Module):

    def __init__(self, nb_hidden):
        super(Net2, self).__init__()
        
        # number of input channels is 2.
        self.conv1 = nn.Conv2d(2, 32, kernel_size = 5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 3)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 1)
        
        
    def forward(self,x):
    
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = 3, stride = 1))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size = 3, stride = 3))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x
    
    
    
def compute_batch_nb_errors(model, input, target, mini_batch_size):
    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        
        output = model(input.narrow(0, b, mini_batch_size))
        predicted_classes = torch.round(torch.sigmoid(output))
        
        for k in range(mini_batch_size):
            if target[b + k] != predicted_classes[k]:
                nb_errors = nb_errors + 1

    return nb_errors


def compute_model_acc(model, input, target):
    nb_errors = 0
    output = model(input)

    for k in range(0, input.size(0)):
        # predict the probability that the first image is lesser or equal to the second one with sigmoid
        if target[k] != torch.round(torch.sigmoid(output[k])):
            nb_errors = nb_errors + 1
            
    return 1-(nb_errors/input.size(0))


    
def train_network(model, train_input, train_target):
    
    # applies the sigmoid and then cross entropy loss     
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr = 1e-1)
    nb_epochs = 50
    mini_batch_size = 100
    
    model.train()
        
    for e in range(nb_epochs):
        
        epochs_loss = 0
        epochs_nb_errors = 0
            
        for b in range(0, train_input.size(0), mini_batch_size):
                
            output = model(train_input.narrow(0, b, mini_batch_size))
            
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size).unsqueeze(1))
            
            nb_errors = compute_batch_nb_errors(model, train_input.narrow(0, b, mini_batch_size), 
                                          train_target.narrow(0, b, mini_batch_size),
                                          mini_batch_size)
            
            model.zero_grad()
            loss.backward()
            optimizer.step()
            
            epochs_loss = epochs_loss + loss.item()
            epochs_nb_errors = epochs_nb_errors + nb_errors
            
        print('Epoch: {:03d} | Loss: {:.02f} | Number Misclassified: {: d}  '.format(
            
            int(e),
            epochs_loss,
            epochs_nb_errors
))

        

In [10]:
mean, std = train_input.mean(), train_input.std()

train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)


model = Net2(200)
print(model)

train_network(model, train_input, train_target)

print('train_accuracy {:.02f}%, test_accuracy {:.02f}%  '.format(
            
            
            compute_model_acc(model, train_input, train_target) * 100,
            compute_model_acc(model, test_input, test_target) * 100
))



Net2(
  (conv1): Conv2d(2, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=1, bias=True)
)
Epoch: 000 | Loss: 6.88 | Number Misclassified:  443  
Epoch: 001 | Loss: 6.74 | Number Misclassified:  431  
Epoch: 002 | Loss: 6.61 | Number Misclassified:  405  
Epoch: 003 | Loss: 6.44 | Number Misclassified:  375  
Epoch: 004 | Loss: 6.22 | Number Misclassified:  341  
Epoch: 005 | Loss: 5.96 | Number Misclassified:  310  
Epoch: 006 | Loss: 5.70 | Number Misclassified:  292  
Epoch: 007 | Loss: 5.47 | Number Misclassified:  277  
Epoch: 008 | Loss: 5.29 | Number Misclassified:  262  
Epoch: 009 | Loss: 5.12 | Number Misclassified:  249  
Epoch: 010 | Loss: 4.96 | Number Misclassified:  239  
Epoch: 011 | Loss: 4.80 | Number Misclassified:  229  
Epoch: 012 | Loss: 4.61 | Number Misclassified:  213  
Epoch: 013 | Loss: 4.49 | Number M